In [ ]:
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = (10,8)
import pylab as plt
import numpy as np
import fitsio
from astrometry.util.fits import *
from astrometry.util.util import Tan
from astrometry.util.plotutils import *
from collections import Counter
from astrometry.util.starutil import *
from scipy.ndimage.filters import *
from scipy.ndimage.measurements import label, find_objects
from scipy.ndimage.morphology import binary_dilation, binary_fill_holes

Read coadded images for making RGB plots

In [ ]:
def sdss_rgb(imgs, bands, scales=None, m=0.03, Q=20):
    rgbscales=dict(g=(2, 6.0),
                   r=(1, 3.4),
                   i=(0, 3.0),
                   z=(0, 2.2))
    if scales is not None:
        rgbscales.update(scales)
    I = 0
    for img,band in zip(imgs, bands):
        plane,scale = rgbscales[band]
        img = np.maximum(0, img * scale + m)
        I = I + img
    I /= len(bands)
    fI = np.arcsinh(Q * I) / np.sqrt(Q)
    I += (I == 0.) * 1e-6
    H,W = I.shape
    rgb = np.zeros((H,W,3), np.float32)
    for img,band in zip(imgs, bands):
        plane,scale = rgbscales[band]
        rgb[:,:,plane] = np.clip((img * scale + m) * fI / I, 0, 1)
    return rgb

In [ ]:
gco = fitsio.read('25/legacysurvey-custom-036450m04600-image-g.fits.fz')
rco = fitsio.read('25/legacysurvey-custom-036450m04600-image-r.fits.fz')
ico = fitsio.read('25/legacysurvey-custom-036450m04600-image-i.fits.fz')
bands = 'gri'
ims = [gco,rco,ico]
s = 4
scale = dict(g=(2, 6.0*s),
            r=(1, 3.4*s),
            i=(0, 3.0*s))
img = sdss_rgb([gco,rco,ico], bands, scales=scale)
rgb = sdss_rgb(ims, bands)
# Show a small portion of the 4000 x 4400 image.
plt.imshow(img[500:1200, 500:1200], origin='lower');

Read the detection maps and inverse-variances

In [ ]:
g_det = fitsio.read('25/detmap-g.fits')
g_detiv = fitsio.read('25/detiv-g.fits')
r_det = fitsio.read('25/detmap-r.fits')
r_detiv = fitsio.read('25/detiv-r.fits')
i_det = fitsio.read('25/detmap-i.fits')
i_detiv = fitsio.read('25/detiv-i.fits')
W,H = g_det.shape
#img = plt.imread('25/legacysurvey-custom-036450m04600-image.jpg')
#img = np.flipud(img)
#H,W,three = img.shape

# In later versions of the code, the WCS headers are correctly written to the detection map images
ra,dec = 36.45, -4.6
pixscale = 0.262 / 3600.
wcs = Tan(ra, dec, W/2.+0.5, H/2.+0.5,
        -pixscale, 0., 0., pixscale,
        float(W), float(H))
imgwcs = wcs

Read the number-of-exposure maps and define a simple "good" region with > 12 exposures per pixel.  These images have a total of 25 exposures included.  This "good" map masks the chip gap, and the edges somewhat.

In [ ]:
Ng = fitsio.read('25/legacysurvey-custom-036450m04600-nexp-g.fits.fz')
Nr = fitsio.read('25/legacysurvey-custom-036450m04600-nexp-r.fits.fz')
Ni = fitsio.read('25/legacysurvey-custom-036450m04600-nexp-i.fits.fz')
good = ((Ng > 12) * (Nr > 12) * (Ni > 12))

In [ ]:
# Compute a SED-matched signal-to-noise map from detection maps and an SED.
def sedsn(detmaps, detivs, sed):
    H,W = detmaps[0].shape
    sedmap = np.zeros((H,W), np.float32)
    sediv  = np.zeros((H,W), np.float32)
    for detmap,detiv,s in zip(detmaps,detivs,sed):
        if s == 0:
            continue
        # We convert the detmap to canonical band via
        #   F_i = detmap_i / sed_i
        # And the corresponding change to sig1 is
        #   sig_f = sig1_i / sed_i
        # And invvar is
        #   invvar_f,i = sed_i^2 / sig1_i^2
        #   invvar_f,i = sed_i^2 * detiv_i
        # So the invvar-weighted accumulator is
        #   F_i * invvar_f,i = (detmap_i / sed_i) * (sed_i^2 * detiv_i)
        #                    = detmap_i * detiv_i * sed_i
        sedmap += detmap * detiv * s
        sediv  += detiv  * s**2
    sedmap /= np.maximum(1e-16, sediv)
    sedsn   = sedmap * np.sqrt(sediv)
    return sedsn

Create our list of SEDs.

In [ ]:
def colorsed(gr, ri):
    return np.array([10.**(-0.4*gr), 1., 10.**(-0.4*-ri)])

class SED(object):
    def __init__(self, name, plotcolor, sed):
        self.name = name
        self.plotcolor = plotcolor
        self.sed = sed
        self.tname = name.replace('-','_') + '_sn'
    def __repr__(self):
        return self.name + ': ' + str(self.sed)

sedlist = [
    SED('blue',   '0.5',    colorsed(0., 0.)),
    SED('yellow', 'orange', colorsed(1., 0.3)),
    SED('red',    'r',      colorsed(1.5, 1.)),
    SED('g-only', 'g',      np.array([1., 0., 0.])),
    SED('r-only', 'pink',   np.array([0., 1., 0.])),
    SED('i-only', 'm',      np.array([0., 0., 1.])),
]
for s in sedlist:
    print('%8s' % s.name, '   '.join(['%6.3f' % x for x in s.sed]))

detmaps = [g_det, r_det, i_det]
detivs = [g_detiv, r_detiv, i_detiv]
for s in sedlist:
    s.snmap = sedsn(detmaps, detivs, s.sed)
#flat_sn = sedlist[0].snmap
yellow_sn = sedlist[1].snmap

In [ ]:
# A simple source detection method that returns the peak pixel within each blob of pixels above threshold.
# No deblending.
def detect_sources(snmap, threshold):
    hot = (snmap > threshold)
    hot = binary_dilation(hot, iterations=2)
    hot = binary_fill_holes(hot)
    blobs,nblobs = label(hot)
    print(nblobs, 'blobs')
    print('blobs min', blobs.min(), 'max', blobs.max())
    slices = find_objects(blobs)
    px,py = [],[]
    for i,slc in enumerate(slices):
        blob_loc = blobs[slc]
        sn_loc = snmap[slc]
        imax = np.argmax((blob_loc == (i+1)) * sn_loc)
        y,x = np.unravel_index(imax, blob_loc.shape)
        y0,x0 = slc[0].start, slc[1].start
        px.append(x0+x)
        py.append(y0+y)
        #if i == 0:
        #    plt.subplot(2,2,1)
        #    plt.imshow(blob_loc, interpolation='nearest', origin='lower')
        #    plt.colorbar()
        #    plt.subplot(2,2,2)
        #    plt.imshow((blob_loc==(i+1))*sn_loc, interpolation='nearest', origin='lower')
        #    plt.subplot(2,2,3)
        #    plt.plot(x, y, 'ro')
    return np.array(px),np.array(py)

In [ ]:
# Run the detection method on the "Yellow" SED-matched filter with a very high threshold.
x,y = detect_sources(yellow_sn, 100.)
sources = fits_table()
sources.x = x
sources.y = y
sources.cut(good[sources.y, sources.x])
print('Cut to', len(sources), 'good sources')
sz = 20
H,W = good.shape
sources.cut((sources.x > sz) * (sources.y > sz) * (sources.x < (W-sz)) * (sources.y < (H-sz)))
print(len(sources), 'not near edges')
sources.cut((g_detiv[sources.y, sources.x] > 0) * (r_detiv[sources.y, sources.x] > 0) * (i_detiv[sources.y, sources.x] > 0))
print(len(sources), 'with gri obs')

for s in sedlist:
    sources.set(s.tname, s.snmap[sources.y, sources.x])

sources.g_sn = (g_det[sources.y, sources.x] * np.sqrt(g_detiv[sources.y, sources.x]))
sources.r_sn = (r_det[sources.y, sources.x] * np.sqrt(r_detiv[sources.y, sources.x]))
sources.i_sn = (i_det[sources.y, sources.x] * np.sqrt(i_detiv[sources.y, sources.x]))
sources.g_flux = g_det[sources.y, sources.x]
sources.r_flux = r_det[sources.y, sources.x]
sources.i_flux = i_det[sources.y, sources.x]
sources.ra,sources.dec = wcs.pixelxy2radec(sources.x+1, sources.y+1)
sources.g_mag = -2.5*(np.log10(sources.g_flux) - 9)
sources.r_mag = -2.5*(np.log10(sources.r_flux) - 9)
sources.i_mag = -2.5*(np.log10(sources.i_flux) - 9)

I = np.argsort(-sources.yellow_sn)
sources.cut(I)

In [ ]:
def show_sources(T, img, R=10, C=10, sz=10, divider=0):
    imgrows = []
    k = 0
    for i in range(R):
        imgrow = []
        for j in range(C):
            if k >= len(T):
                sub = np.zeros((sz*2+1,sz*2+1,3), np.uint8)
            else:
                f = T[k]
                sub = img[f.y-sz : f.y+sz+1, f.x-sz : f.x+sz+1, :]
            imgrow.append(sub)
            if divider and j < C-1:
                imgrow.append(np.zeros((sz*2+1,divider,3), np.uint8)+255)
            k += 1
        imgrow = np.hstack(imgrow)
        #print('imgrow', imgrow.shape)
        imgrows.append(imgrow)
        if divider and i < R-1:
            rh,rw,three = imgrow.shape
            imgrows.append(np.zeros((divider, rw, 3), np.uint8) + 255)
        
    imgrows = np.vstack(reversed(imgrows))
    plt.imshow(imgrows, interpolation='nearest', origin='lower')
    plt.xticks([]); plt.yticks([]);

In [ ]:
show_sources(sources, img)

In [ ]:
sources.imax = np.argmax(np.vstack([sources.get(s.tname) for s in sedlist]), axis=0)

Where in color-color space do sources detected most strongly by each of the SED-matched filters live?

In [ ]:
for i,s in enumerate(sedlist):
    sed = s.sed
    if not np.all(sed > 0):
        continue
    I = np.flatnonzero(sources.imax == i)
    plt.plot(sources.g_mag[I] - sources.r_mag[I], sources.r_mag[I] - sources.i_mag[I], '.', label=s.name,
            color=s.plotcolor)
    gr = -2.5 * np.log10(sed[0] / sed[1])
    ri = -2.5 * np.log10(sed[1] / sed[2])
    #print(sedlist[i].name, 'gr', gr, 'ri', ri)
    plt.plot(gr, ri, 'ko', mfc='none', ms=8, mew=3)
plt.axis([-0.8, 2.5, -0.8, 2])
plt.xlabel('g-r (mag)')
plt.ylabel('r-i (mag)')
plt.legend();
plt.savefig('best-color.png')

Show sources that are most strongly detected by each of the SED-matched filters.

In [ ]:
for i,s in enumerate(sedlist):
    I = np.flatnonzero(sources.imax == i)
    #print(len(I), s.name)
    show_sources(sources[I], img)
    plt.title('%s best (%i)' % (s.name, len(I)));
    plt.savefig('best-%s.png' % s.name)
    #print('Coords:', list(zip(sources.x[I], 4400-sources.y[I]))[:10])
    plt.show();

Demonstrate galaxy detection

In [ ]:


# do 1 last so that g_galdet etc keep those values
#for galsig in [2, 1]:
#re = 0.7
#s = 're%.1f' % re
#if True:
for galsig in [1]:
    s = '%.1f' % galsig
    g_galdet = fitsio.read('25/galdetmap-'+s+'-g.fits')
    g_galdetiv = fitsio.read('25/galdetiv-'+s+'-g.fits')
    r_galdet = fitsio.read('25/galdetmap-'+s+'-r.fits')
    r_galdetiv = fitsio.read('25/galdetiv-'+s+'-r.fits')
    i_galdet = fitsio.read('25/galdetmap-'+s+'-i.fits')
    i_galdetiv = fitsio.read('25/galdetiv-'+s+'-i.fits')
    gdetmaps = [g_galdet, r_galdet, i_galdet]
    gdetivs = [g_galdetiv, r_galdetiv, i_galdetiv]
    for s in sedlist:
        setattr(s, 'galsnmap_%i' % int(galsig), sedsn(gdetmaps, gdetivs, s.sed))
        #s.galsnmap = sedsn(gdetmaps, gdetivs, s.sed)

yellow_gal = sedlist[1].galsnmap_1
#yellow_gal = sedlist[1].galsnmap
x,y = detect_sources(yellow_gal, 100.)

gals = fits_table()
gals.x = x
gals.y = y
gals.cut(good[gals.y, gals.x])
print('Cut to', len(gals), 'good gals')
sz = 20
H,W = good.shape
gals.cut((gals.x > sz) * (gals.y > sz) * (gals.x < (W-sz)) * (gals.y < (H-sz)))
print(len(gals), 'not near edges')
gals.cut((g_galdetiv[gals.y, gals.x] > 0) * (r_galdetiv[gals.y, gals.x] > 0) * (i_galdetiv[gals.y, gals.x] > 0))
print(len(gals), 'with gri obs')

sns = []
for s in sedlist:
    gals.set(s.tname, s.galsnmap_1[gals.y, gals.x])
    #gals.set(s.tname+'_2', s.galsnmap_2[gals.y, gals.x])
    #gals.set(s.tname, s.galsnmap[gals.y, gals.x])
    gals.set(s.tname+'_psf', s.snmap[gals.y, gals.x])
    sns.append(s.galsnmap[gals.y, gals.x])
gals.sn_max = np.max(np.vstack(sns), axis=0)
    
# These are PSF fluxes/mags
gals.g_sn = (g_det[gals.y, gals.x] * np.sqrt(g_detiv[gals.y, gals.x]))
gals.r_sn = (r_det[gals.y, gals.x] * np.sqrt(r_detiv[gals.y, gals.x]))
gals.i_sn = (i_det[gals.y, gals.x] * np.sqrt(i_detiv[gals.y, gals.x]))
gals.g_flux = g_det[gals.y, gals.x]
gals.r_flux = r_det[gals.y, gals.x]
gals.i_flux = i_det[gals.y, gals.x]
gals.ra,gals.dec = wcs.pixelxy2radec(gals.x+1, gals.y+1)
gals.g_mag = -2.5*(np.log10(gals.g_flux) - 9)
gals.r_mag = -2.5*(np.log10(gals.r_flux) - 9)
gals.i_mag = -2.5*(np.log10(gals.i_flux) - 9)

gals.g_galflux = g_galdet[gals.y, gals.x]
gals.r_galflux = r_galdet[gals.y, gals.x]
gals.i_galflux = i_galdet[gals.y, gals.x]
gals.g_galmag = -2.5*(np.log10(gals.g_galflux) - 9)
gals.r_galmag = -2.5*(np.log10(gals.r_galflux) - 9)
gals.i_galmag = -2.5*(np.log10(gals.i_galflux) - 9)

I = np.argsort(-gals.yellow_sn)
gals.cut(I)    

In [ ]:
gals.imax = np.argmax(np.vstack([gals.get(s.tname) for s in sedlist]), axis=0)

In [ ]:
plt.figure(figsize=(10,10))
I = np.argsort(-gals.yellow_sn / gals.yellow_sn_psf)
show_sources(gals[I], img, sz=20)

In [ ]:
plt.figure(figsize=(10,10))
I = np.argsort(-(gals.yellow_sn - gals.yellow_sn_psf))
show_sources(gals[I], img, sz=20)
#plt.savefig('gals-exp.png')
plt.savefig('gals-gauss.png')

In [ ]:
plt.figure(figsize=(10,10))
I = np.argsort(-(gals.yellow_sn - gals.yellow_sn_psf) * (gals.yellow_sn == gals.sn_max))
show_sources(gals[I], img, sz=20)

In [ ]:
plt.figure(figsize=(10,10))
I = np.argsort(-(gals.red_sn - gals.red_sn_psf) * (gals.red_sn == gals.sn_max))
show_sources(gals[I], img, sz=15)

In [ ]:
plt.figure(figsize=(10,10))
#I = np.argsort(-(gals.blue_sn - gals.blue_sn_psf) * (gals.blue_sn == gals.sn_max))
f = gals.yellow_sn / gals.yellow_sn_psf
#I = np.flatnonzero((f > 0.97) * (f < 1))
#I = np.flatnonzero((f > 1.05) * (f < 1.06))
#I = np.flatnonzero((f > 0.93) * (f < 0.97))
I = np.flatnonzero((f > 1.1) * (f < 1.15))
I = I[np.argsort(-gals.yellow_sn[I])]
show_sources(gals[I], img, sz=15)

In [ ]:
plt.plot(gals.yellow_sn, gals.yellow_sn_psf, 'b.')
plt.plot([100,20000], [100,20000], 'k-')
plt.xscale('log')
plt.yscale('log')

In [ ]:
plt.plot(gals.yellow_sn, gals.yellow_sn - gals.yellow_sn_psf, 'b.')
plt.xscale('log')
#plt.yscale('log')

In [ ]:
plt.semilogx(gals.yellow_sn, gals.yellow_sn / gals.yellow_sn_psf, 'k.', alpha=0.5);
plt.ylim(0.9, 1.2)
#plt.savefig('relsn-exp.png')
#plt.savefig('relsn-gauss.png')

In [ ]:
#for i,s in reversed(list(enumerate(sedlist))):
for i,s in enumerate(sedlist):
    sed = s.sed
    if not np.all(sed > 0):
        continue
    I = np.flatnonzero(gals.imax == i)
    #plt.semilogx(gals.yellow_sn[I], gals.yellow_sn[I] / gals.yellow_sn_psf[I], 'k.');
    plt.semilogx(gals.get(s.tname)[I], gals.get(s.tname)[I] / gals.get(s.tname + '_psf')[I],
                 '.', label=s.name, color=s.plotcolor, alpha=0.5)
plt.ylim(0.9, 1.2)
plt.legend();
#plt.savefig('best-color.png')

In [ ]:
#np.sum(gals.yellow_sn > gals.yellow_sn_psf), len(gals)

In [ ]:
plt.plot(gals.g_galmag - gals.r_galmag, gals.r_galmag - gals.i_galmag, 'k.')
for i,s in enumerate(sedlist):
    sed = s.sed
    if not np.all(sed > 0):
        continue
    I = np.flatnonzero((gals.imax == i) * (gals.get(s.tname) > gals.get(s.tname+'_psf')))
    plt.plot(gals.g_galmag[I] - gals.r_galmag[I], gals.r_galmag[I] - gals.i_galmag[I],
         '.', label=s.name, color=s.plotcolor, alpha=0.5)
plt.axis([-0.5, 2.5, -.5, 2])
plt.xlabel('g-r (mag)')
plt.ylabel('r-i (mag)');
#I = np.flatnonzero(np.isfinite(gals.g_galmag) * np.isfinite(gals.r_galmag) * np.isfinite(gals.i_galmag))
#print('median g-r:', np.median(gals.g_galmag[I] - gals.r_galmag[I]), 'r-i:', np.median(gals.r_galmag[I] - gals.i_galmag[I]))

In [ ]:
plt.plot(gals.g_galmag - gals.r_galmag, gals.r_galmag - gals.i_galmag, 'k.', alpha=0.25)
plt.axis([-0.5, 2.5, -.5, 2])
plt.xlabel('g-r (mag)')
plt.ylabel('r-i (mag)');
I = np.flatnonzero(np.isfinite(gals.g_galmag) * np.isfinite(gals.r_galmag) * np.isfinite(gals.i_galmag))
print('median g-r:', np.median(gals.g_galmag[I] - gals.r_galmag[I]), 'r-i:', np.median(gals.r_galmag[I] - gals.i_galmag[I]))

In [ ]:
plt.imshow(img[500:1000, 500:1000]);

In [ ]:
plt.plot(sources.g_mag - sources.r_mag, sources.r_mag - sources.i_mag, 'k.', alpha=0.25)
plt.axis([-0.5, 2.5, -.5, 2])
plt.xlabel('g-r (mag)')
plt.ylabel('r-i (mag)');
I = np.flatnonzero(np.isfinite(sources.g_mag) * np.isfinite(sources.r_mag) * np.isfinite(sources.i_mag))
print('median g-r:', np.median(sources.g_mag[I] - sources.r_mag[I]), 'r-i:', np.median(sources.r_mag[I] - sources.i_mag[I]))

In [ ]:
plt.imshow(yellow_sn, interpolation='nearest', origin='lower', vmin=0, vmax=50, cmap='gray')
ax = plt.axis()
plt.plot(sources.x, sources.y, 'r.')
plt.axis(ax)
plt.axis([0,500,0,500]);

In [ ]:
plt.subplots_adjust(hspace=0.01, wspace=0.01)
N=100
R,C = 9,12
for i in range(len(sources)):
    if i >= R*C:
        break
    plt.subplot(R,C,i+1)
    f = sources[i]
    plt.imshow(img[f.y-sz : f.y+sz+1, f.x-sz : f.x+sz+1, :], interpolation='nearest', origin='lower')
    plt.xticks([]); plt.yticks([])    

In [ ]:
show_sources(sources, img, R=6, C=6, divider=2)

In [ ]:
I = np.argsort(-sources.red_sn / sources.flat_sn)
show_sources(sources[I], img)

In [ ]:
I = np.argsort(-sources.blue_sn / sources.flat_sn)
show_sources(sources[I], img)

In [ ]:
sources.imax = np.argmax(np.vstack([sources.get(s.tname) for s in sedlist]), axis=0)

In [ ]:
I = np.hstack((np.flatnonzero(sources.imax == 3),
              np.flatnonzero(sources.imax == 4),
              np.flatnonzero(sources.imax == 5)))
show_sources(sources[I], img, R=6, C=7)

In [ ]:
img.shape

In [ ]:
from astrometry.util.util import wcs_pv2sip_hdr
ccds = fits_table('survey-ccds-snx3-25.fits.gz')
ccds.cut(np.argsort(ccds.mjd_obs))
wcses = []
Fhdu = []
for ccd in ccds:
    #F = fitsio.FITS(ccd.image_filename.strip())
    hdr = fitsio.read_header(ccd.image_filename.strip(), ext=ccd.ccdname.strip())
    wcs = wcs_pv2sip_hdr(hdr)
    wcses.append((wcs, ccd))
    Fhdu.append(fitsio.FITS(ccd.image_filename.strip())[ccd.ccdname.strip()])
#plt.plot(sources.x[I], sources.y[I])


for isrc,s in enumerate(sources[I]):
    if not isrc+1 in [4, 6, 9, 10]: #[1, 8, 29]:
        continue
    plt.imshow(img[s.y-50:s.y+51, s.x-50:s.x+51,:])
    plt.show()
    imgs = []
    for j,(wcs,ccd) in enumerate(wcses):
        ok,xx,yy = wcs.radec2pixelxy(s.ra, s.dec)
        if not ((xx >= 1) * (yy >= 1) * (xx < wcs.get_width()) * (yy < wcs.get_height())):
            continue
        x = int(xx)-1
        y = int(yy)-1
        sz = 50
        subimg = Fhdu[j][y-sz:y+sz+1, x-sz:x+sz+1]
        imgs.append((subimg,ccd))
    print(len(imgs))
    plt.figure(figsize=(10,10))
    plt.subplots_adjust(left=0.01, right=0.99, bottom=0.01, top=0.99, hspace=0.2, wspace=0.1)
    plt.clf()
    for k,(subimg,ccd) in enumerate(imgs[:81]):
        plt.subplot(8,10,k+1)
        pp = np.percentile(subimg.ravel(), [16,50,84])
        med = pp[1]
        sig = (pp[2]-pp[0])/2.
        plt.imshow(subimg, vmin=med-3.*sig, vmax=med+10.*sig)
        if subimg[sz,sz] > med+5.*sig:
            plt.title('%i' % ccd.expnum, color='red')
            print(isrc+1, 'jpl_query(%s, %s, %s)' % (s.ra, s.dec, ccd.mjd_obs))
        else:
            plt.title('%i' % ccd.expnum)
        plt.xticks([]); plt.yticks([])
    plt.show();


In [ ]:
plt.plot(sources.red_sn / sources.flat_sn, 'r.');

In [ ]:
wcs = imgwcs
DES = fits_table('des-db-2.fits')
print(len(DES))
DES.cut((DES.mag_auto_g < 99) * (DES.mag_auto_r < 99) * (DES.mag_auto_i < 99))
print(len(DES))
ok,x,y = wcs.radec2pixelxy(DES.ra, DES.dec)
DES.x = (x-1).astype(np.int)
DES.y = (y-1).astype(np.int)
DES.cut((DES.x > sz) * (DES.y > sz) * (DES.x < (W-sz)) * (DES.y < (H-sz)))
print(len(DES))
DES.cut((g_detiv[DES.y, DES.x] > 0) * (r_detiv[DES.y, DES.x] > 0) * (i_detiv[DES.y, DES.x] > 0))
print(len(DES))

In [ ]:
plt.hist(DES.mag_auto_g);

In [ ]:
MI,MJ,d = match_radec(sources.ra, sources.dec, DES.ra, DES.dec, 1./3600, nearest=True)
print(len(MI), len(sources))
MDES = DES[MJ]
Msources = sources[MI]

In [ ]:
ha = dict(range=(20,28), bins=50, histtype='step')
plt.hist(DES.mag_auto_r, **ha);
plt.hist(MDES.mag_auto_r, **ha);

In [ ]:
#K = np.flatnonzero(DES.mag_auto_r < 23.)
K = np.flatnonzero(DES.mag_auto_r < 27.)
print(len(K))
#plt.plot(DES.mag_auto_g[K] - DES.mag_auto_r[K], DES.mag_auto_r[K] - DES.mag_auto_i[K], 'k.')
#plt.axis([-0.5, 3, -0.5, 2]);
plothist(DES.mag_auto_g[K] - DES.mag_auto_r[K], DES.mag_auto_r[K] - DES.mag_auto_i[K],
         range=((-0.5, 3),(-0.5,2)));

In [ ]:
DES.flux_g = 10. ** ((DES.mag_auto_g - 22.5) / -2.5)
DES.flux_r = 10. ** ((DES.mag_auto_r - 22.5) / -2.5)
DES.flux_i = 10. ** ((DES.mag_auto_i - 22.5) / -2.5)
flux = DES.flux_g + DES.flux_r + DES.flux_i
DES.f_g = DES.flux_g / flux
DES.f_r = DES.flux_r / flux
DES.f_i = DES.flux_i / flux

In [ ]:
N,xe,ye = loghist(DES.f_g[K], DES.f_r[K], range=((0,1),(0,1)), nbins=10);

In [ ]:
np.sum(N > 0), N.max()

In [ ]:
plt.hist(DES.f_g[K], histtype='step', color='g');
plt.hist(DES.f_r[K], histtype='step', color='r');
plt.hist(DES.f_i[K], histtype='step', color='m');

In [ ]:
iy,ix = np.nonzero(N)
# Find f_{g,r} histogram midpoints
mx = (xe[:-1] + xe[1:]) / 2.
my = (ye[:-1] + ye[1:]) / 2.
fx = mx[ix]
fy = my[iy]
fn = N[iy,ix]

In [ ]:
plt.scatter(fx, fy, c=np.log10(fn), vmax=np.log10(N.max())+1);

In [ ]:
fg = fx
fr = fy
fi = 1. - (fg + fr)
seds = np.clip(np.vstack((fg,fr,fi)).T, 0., 1.)
weights = fn / np.sum(fn)

In [ ]:
seds.shape, weights.shape, weights.dtype

In [ ]:
from scipy.special import erf, erfc, erfcx
#def p_bayes(seds, weights, D, sig1, alpha):
def pratio_bayes(seds, weights, D, Div, alpha):
    '''
    N: # SEDs
    J: # bands
    H,W: # pixels

    seds: N x J
    weights: N
    D: J x H x W
    sig1: J
    Div: J x H x W
    '''
    J,H,W = D.shape
    N = len(weights)
    assert(seds.shape == (N,J))
    assert(weights.shape == (N,))
    #assert(sig1.shape == (J,))
    assert(Div.shape == (J,H,W))

    psum = 0.
    for i in range(N):
        # sum over j
        #a_i = alpha - np.sum(D * seds[i,:,np.newaxis,np.newaxis] / sig1[:,np.newaxis,np.newaxis]**2, axis=0)
        a_i = alpha - np.sum(D * seds[i,:,np.newaxis,np.newaxis] * Div, axis=0)
        assert(a_i.shape == (H,W))

        # sum over j
        #b_i = 0.5 * np.sum(seds[i,:]**2 / sig1**2)
        #assert(np.isscalar(b_i))
        b_i = 0.5 * np.sum(seds[i,:,np.newaxis,np.newaxis]**2 * Div, axis=0)
        assert(b_i.shape == (H,W))
        beta_i = 2 * np.sqrt(b_i)
        c_i = a_i / beta_i

        # erfc(x) is 1 - erf(x)
        # erfcx(x) is exp(x^2) * (1-erf(x))
        #psum += weights[i] / beta_i * np.exp(c_i**2) * erfc(c_i)
        psum += weights[i] / beta_i * erfcx(c_i)

    #K = 1./np.prod(np.sqrt(2.*np.pi)*sig1) * np.exp(-0.5 * np.sum((D / sig_j[:,np.newaxis,np.newaxis])**2, axis=0))
    #K = np.prod(Div, axis=0)/(np.sqrt(2.*np.pi)**J) * np.exp(-0.5 * np.sum(D**2 * Div, axis=0))
    #prob = K * alpha * np.sqrt(np.pi) * psum
    pr = alpha * np.sqrt(np.pi) * psum
    return pr

In [ ]:
H,W = detmaps[0].shape
J = 3
D = np.zeros((J,H,W))
Div = np.zeros((J,H,W))
for i,(d,div) in enumerate(zip(detmaps,detivs)):
    D[i,:,:] = d
    Div[i,:,:] = div
alpha = 1.
prb = pratio_bayes(seds, weights, D, Div, alpha)

In [ ]:
print(np.sum(np.logical_not(np.isfinite(prb))))
prb[np.logical_not(good)] = 0.
print(np.sum(np.logical_not(np.isfinite(prb))))
prb[np.logical_not(np.isfinite(prb))] = 0.

In [ ]:
plt.figure(figsize=(6,6))
#plt.imshow(np.log10(prb[:2000,:2000]));
plt.imshow(prb, vmin=0, vmax=1e3)
plt.colorbar();

In [ ]:
plt.imshow(np.logical_not(np.isfinite(prb)));

In [ ]:
plt.hist(np.log10(prb.ravel()), log=True, bins=20, range=(-5,20));

In [ ]:
xg,yg = detect_sources(detmaps[0] * np.sqrt(detivs[0]), 50.)
xr,yr = detect_sources(detmaps[1] * np.sqrt(detivs[1]), 50.)
xi,yi = detect_sources(detmaps[2] * np.sqrt(detivs[2]), 50.)

In [ ]:
xm,ym = xg.copy(),yg.copy()
for xx,yy in [(xr,yr),(xi,yi)]:
    I,J,d = match_xy(xm,ym, xx,yy, 5.)
    print('Matched:', len(I))
    U = np.ones(len(xx), bool)
    U[J] = False
    print('Unmatched:', np.sum(U))
    xm = np.hstack((xm, xx[U]))
    ym = np.hstack((ym, yy[U]))

In [ ]:
sources = fits_table()
sources.x = xm
sources.y = ym

iy,ix = np.round(sources.y).astype(int), np.round(sources.x).astype(int)
sources.sn_g = (detmaps[0] * np.sqrt(detivs[0]))[iy,ix]
sources.sn_r = (detmaps[1] * np.sqrt(detivs[1]))[iy,ix]
sources.sn_i = (detmaps[2] * np.sqrt(detivs[2]))[iy,ix]
sources.sn_max = np.maximum(sources.sn_g, np.maximum(sources.sn_r, sources.sn_i))
sources.cut((sources.x > sz) * (sources.x < (W-sz)) * (sources.y > sz) * (sources.y < (H-sz)))
print('Kept', len(sources))

In [ ]:
I = np.argsort(-sources.sn_max)
show_sources(sources[I], img)

In [ ]:
bx,by = detect_sources(prb, 1e200)

bsources = fits_table()
bsources.x = bx
bsources.y = by

iy,ix = np.round(bsources.y).astype(int), np.round(bsources.x).astype(int)
bsources.prb = prb[iy,ix]
bsources.cut((bsources.x > sz) * (bsources.x < (W-sz)) * (bsources.y > sz) * (bsources.y < (H-sz)))
print('Kept', len(bsources))
plt.figure(figsize=(8,8))
I = np.argsort(-bsources.prb)
show_sources(bsources[I], img, R=20, C=20)

In [ ]:
np.sum(prb > 2e-6)

In [ ]:
4400 * 4000

In [ ]:
np.sum(fn)
seds

In [ ]:
#plt.plot(DES.mag_auto_g - DES.mag_auto_r, DES.mag_auto_r - DES.mag_auto_i, 'r.')
plt.plot(MDES.mag_auto_g - MDES.mag_auto_r, MDES.mag_auto_r - MDES.mag_auto_i, 'k.')
plt.axis([-0.5, 3, -0.5, 2]);

In [ ]:
## FIXME -- star vs galaxy; isolated
colorbins = np.linspace(-0.5, 4.0, 10)
#colorbins = np.linspace(-0.5, 4.0, 19)
II = []
K = []
DES.gi = DES.mag_auto_g - DES.mag_auto_i
for clo,chi in zip(colorbins, colorbins[1:]):
    C = np.flatnonzero((DES.gi >= clo) * (DES.gi < chi))
    minmag = np.vstack((DES.mag_auto_g, DES.mag_auto_r, DES.mag_auto_i)).max(axis=0)[C]
    #I.extend(J[np.argsort(DES.mag_auto_r[J])[:10]])
    C = C[np.argsort(np.abs(minmag - 17.9))]
    II.extend(C[:10])
    K.append(C[0])
    #print('min mags', np.sort(minmag)[:10])
    #print('r mags', np.sort(DES.mag_auto_r[J])[:10])
show_sources(DES[II], img)

In [ ]:
plt.axhline(1., color='orange', lw=5)
plt.axhline(1., color='k', alpha=0.5)
plt.plot(MDES.mag_auto_g - MDES.mag_auto_i, Msources.blue_sn / Msources.yellow_sn, 'bx', alpha=0.3,
        label='Blue SED');
plt.plot(MDES.mag_auto_g - MDES.mag_auto_i, Msources.red_sn  / Msources.yellow_sn, 'r.', alpha=0.5,
        label='Red SED');
plt.xlabel('DES g-i color (mag)')
plt.ylabel('Relative strength of SED-matched filter vs Yellow');
plt.legend(loc='upper left')
ymin = 0.5
plt.axis([-0.5, 4.0, ymin, 1.3])
ax = plt.axis()
aspect = plt.gca().get_aspect()
for clo,chi,k in zip(colorbins, colorbins[1:], K):
    x,y = DES.x[k], DES.y[k]
    plt.imshow(img[y-sz:y+sz+1, x-sz:x+sz+1], interpolation='nearest', origin='lower',
              extent=[clo,chi,ymin,ymin+0.15], zorder=20)
plt.axis(ax)
plt.gca().set_aspect(aspect);
plt.savefig('strength3.png')

In [ ]:
# Galaxy detection.
from tractor.splinesky import SplineSky
from tractor.psfex import PixelizedPsfEx
from astrometry.util.util import wcs_pv2sip_hdr

imfn = '1/data/images/decam/cp/c4d_160814_085515_ooi_g_v1-N4.fits'
im = fitsio.read(imfn)                 
dq = fitsio.read('1/data/images/decam/cp/c4d_160814_085515_ood_g_v1-N4.fits')
wt = fitsio.read('1/data/images/decam/cp/c4d_160814_085515_oow_g_v1-N4.fits')
sig1 = 1./np.sqrt(np.median(wt[dq==0]))
H,W = im.shape

hdr = fitsio.read_header(imfn, ext=1)
imwcs = wcs_pv2sip_hdr(hdr)

fn = '1/data/calib/decam/splinesky/00563/00563982/decam-00563982-N4.fits'
sky = SplineSky.from_fits(fn, None)
sky.addTo(im, scale=-1)
psf = PixelizedPsfEx('1/data/calib/decam/psfex/00563/00563982/decam-00563982-N4.fits')

In [ ]:
#plt.hist(im.ravel(), range=(-5.*sig1, 5.*sig1), bins=100);

In [ ]:
from tractor import Image, NullWCS, ExpGalaxy, ConstantSky, LinearPhotoCal, NanoMaggies, PixPos, GaussianMixturePSF, Tractor
from tractor.ellipses import EllipseE

In [ ]:
v = (psf.fwhm / 2.35)**2
gpsf = GaussianMixturePSF(1., 0., 0., v, v, 0)                          
tim = Image(data=im, inverr=(dq == 0)*1./sig1, wcs=NullWCS(pixscale=0.262),
           psf=psf, sky=ConstantSky(0.), photocal=LinearPhotoCal(1., band='g'))

In [ ]:
gal = ExpGalaxy(PixPos(W/2., H/2.), NanoMaggies(g=1.), EllipseE(0.7, 0., 0.))
mog = gal._getAffineProfile(tim, W/2., H/2.)
#print(mog)
mog.var[:,0,0]

In [ ]:
list(zip(ExpGalaxy.profile.amp, np.sqrt(ExpGalaxy.profile.var[:,0,0] / (0.262/0.7)**2)))

In [ ]:
#tr = Tractor([tim], [gal])
mod = gal.getModelPatch(tim)
tim.psf = gpsf
gmod = gal.getModelPatch(tim)
print(mod.patch.min(), mod.patch.max())
print(gmod.patch.min(), gmod.patch.max())
plt.subplot(1,2,1)
plt.imshow(mod.patch, interpolation='nearest', origin='lower')
plt.subplot(1,2,2)
plt.imshow(gmod.patch, interpolation='nearest', origin='lower')

In [ ]:
psf_sigma = psf.fwhm / 2.35
print('PSF sigma:', psf_sigma)

gpsf = gmod.patch
print('gal x psf sum', gpsf.sum())
gpsf /= gpsf.sum()
gpsfnorm = np.sqrt(np.sum(gpsf**2))
print('gal x psf norm', gpsfnorm)

gdetsum = 0.
for amp,sigma in zip(mog.amp, np.sqrt(mog.var[:,0,0])):
    sig = np.hypot(psf_sigma, sigma)
    gdetsum = gdetsum + amp * gaussian_filter(im, sig)
    print(gdetsum.shape)
gdetsum /= gpsfnorm**2
gdetsig = sig1 / gpsfnorm

# PSF detection map
psfnorm = 1./(2.*np.sqrt(np.pi)*psf_sigma)
print('PSF norm', psfnorm)
psfdet = gaussian_filter(im, psf_sigma) / psfnorm**2
psfsig1 = sig1 / psfnorm

In [ ]:
psfsn = psfdet / psfsig1
galsn = gdetsum / gdetsig

plt.subplot(1,2,1)
plt.imshow(psfsn, interpolation='nearest', origin='lower', vmin=-3, vmax=10.)
plt.subplot(1,2,2)
plt.imshow(galsn, interpolation='nearest', origin='lower', vmin=-3, vmax=10.);

In [ ]:
psf = fits_table()
# x,y in detection image
psf.dx,psf.dy = detect_sources(psfsn, 10.)
gal = fits_table()
gal.dx, gal.dy = detect_sources(galsn, 10.)

psf.psf_sn = psfsn[psf.dy, psf.dx]
psf.gal_sn = galsn[psf.dy, psf.dx]
gal.psf_sn = psfsn[gal.dy, gal.dx]
gal.gal_sn = galsn[gal.dy, gal.dx]
print(len(psf), 'PSF detections')
print(len(gal), 'Galaxy detections')

In [ ]:
# for viewing, convert to x,y in RGB image.
r,d = imwcs.pixelxy2radec(psf.dx+1, psf.dy+1)
ok,x,y = wcs.radec2pixelxy(r, d)
psf.x = (x-1).astype(int)
psf.y = (y-1).astype(int)

r,d = imwcs.pixelxy2radec(gal.dx+1, gal.dy+1)
ok,x,y = wcs.radec2pixelxy(r, d)
gal.x = (x-1).astype(int)
gal.y = (y-1).astype(int)

H,W,nil = img.shape
galok = gal[(gal.x > sz) * (gal.y > sz) * (gal.x < (W-sz)) * (gal.y < (H-sz))]
#galok.cut((g_detiv[galok.y, galok.x] > 0) * (r_detiv[galok.y, galok.x] > 0) * (i_detiv[galok.y, galok.x] > 0))

#I = np.argsort(-galok.gal_sn / galok.psf_sn);
I = np.argsort(-(galok.gal_sn - galok.psf_sn))
show_sources(galok[I], img)

In [ ]:
S = fits_table('sweep-240p005-250p010.fits')
len(S)

In [ ]:
S.mag_g = -2.5*(np.log10(S.flux_g)-9)
S.mag_r = -2.5*(np.log10(S.flux_r)-9)
S.mag_z = -2.5*(np.log10(S.flux_z)-9)

In [ ]:
plothist(S.mag_g - S.mag_r, S.mag_r - S.mag_z, range=((-1,5),(-1,5)));
#plt.axis([-1, 5, -1, 5])


In [ ]:
E, = np.nonzero([t[0] == 'E' for t in S.type])
plt.hist(S.shapeexp_r[E], bins=50, range=(0,3));
np.median(S.shapeexp_r[E])
plt.axvline(0.7)